In [51]:
import os
import itertools
import MeCab
import pandas as pd
import csv

import work

from tqdm import tqdm
from collections import defaultdict
from collections import OrderedDict

In [2]:
path = './tsv/front-512/42'

train = work.load_tsv_dataset(f'{path}/train.tsv')
valid = work.load_tsv_dataset(f'{path}/dev.tsv')
test = work.load_tsv_dataset(f'{path}/test.tsv')
works = train + valid + test

print(f'num. of train: {len(train)}')
print(f'num. of valid: {len(valid)}')
print(f'num. of test : {len(test)}')
print(f'num. of all  : {len(works)}')

num. of train: 2963
num. of valid: 370
num. of test : 371
num. of all  : 3704


In [3]:
del(train)
del(valid)
del(test)

In [3]:
for work in works[:10]:
    if work['label'] == 0:
        print(work['label'])

0
0


In [4]:
pos_works = [work for work in works if not work['label']==0]

In [5]:
for work in pos_works[:50]:
    if work['label'] == 0:
        print(work['label'])

In [6]:
print(len(pos_works))

1852


In [7]:
print(pos_works[:5])

[{'label': 1, 'content': '赤き月が照らす夜、『鉄血』の真祖であるフリーダは静かに月を眺めていた。彼女は自分の人生に飽き飽きしていた。真祖として至上の力を持つ彼女は、一度として命の危険を感じたこともなかったし、欲しいものは力をふるえばすぐ手に入った。彼女は常に──『退屈』していたのだ。「──フリーダ様！」そんな折、唯一の眷族である『エルマ』が羽を広げて飛んできた。強い力を持つ彼女は配下も小間使いの一人だけで十分だった。「何用だ？私は今、月を眺めて退屈な時間を紛らわしているところだ。何百何千年と変わらぬ日々。私は何のために生まれてきたのかと常に考えさせられる」「それはもちろんこの世界の『支配』者とし「早く要件を言え」フリーダにきつく言われ、エルマはすごすごと持ってきたものを差し出す。「支配のことなんぞネロに任せておけ……ん？」それはゆりかごに包まれた赤ん坊だった。白い髪に白い肌。目は鮮血のように赤い。「……『忌血』の子供か」「はい、人間たちは忌血は災いや良くないものを引き寄せるとして忌み嫌っているそうで、この捨て子が、近くを流れている川の沿岸に漂着していました。おやつにどうかなって、忌血の血はものすっごく美味しいんですよ！」フ１４歳の少年'}, {'label': 1, 'content': 'この世界に転生してからずっと待ち望んでいた非科学的存在。それは深夜、妹の部屋に現れた。俺の存在に気付かず、ベッドの上の妹に声をかけている。水色の熊のぬいぐるみ。生き物には見えないし、正気ならそんな物が喋って動いていたら寝惚けていると思うだろう。俺はぬいぐるみに向かって走り出し。「僕と契約して魔法少女に、、、」「ダメに決まってんだろー！！！」そいつを踏み潰した。「兄さん！！？」★いい歳したおっさんが自分の部屋で1人、ゲームをプレイしている。夜が明けて日が昇り始めても、手を止めることなくずっとだ。ボサボサに伸びきった髪が目にかかって見えにくいが、気にしない。切るのも面倒くさいしな。服は上下スウェットのまま、もう何日着替えていないのか覚えていない。誰かが、扉をノックする音が聞こえる。「、、、」ゲームをプレイする手を止めずに、チラりと時計を見る。時計の針は午前10時を指していた。この時間に俺の部屋をノックする人はこの家に1人しかいない。扉越しに声が聞こえてくる。

In [9]:
def analyze(n, v, a, text):
    noun_list = []
    verb_list = []
    adjv_list = []
    mecab = MeCab.Tagger("")
    
    node = mecab.parseToNode(text)
    while node:
        term = node.feature.split(",")[0]
        #print(f'{node.surface}:{term}')
        if term == "名詞":
            noun_list.append(node.surface)
        elif term == "動詞":
            verb_list.append(node.surface)
        elif term == "形容詞" or term == "副詞":
            adjv_list.append(node.surface)
        else:
            pass
        node = node.next
    
    for noun in noun_list:
        n[noun] += 1
            
    for verb in verb_list:
        v[verb] += 1
            
    for adjv in adjv_list:
        a[adjv] += 1
    
    return n, v, a
    

In [49]:
noun = defaultdict(int)
verb = defaultdict(int)
adjv = defaultdict(int)

"""
noun = pd.DataFrame(columns = ['word', 'count'])
verb = pd.DataFrame(columns = ['word', 'count'])
adjv = pd.DataFrame(columns = ['word', 'count'])
""" 
for work in tqdm(pos_works):
    sep = work['content'].split("。")
    for text in sep:
        noun, verb, adjv = analyze(noun, verb, adjv, text)
"""
print(f'名詞　: {noun}')
print(f'動詞　: {verb}')
print(f'形容詞: {adjv}')
"""

100%|███████████████████████████████████████████████████████████████████████████████| 1852/1852 [00:52<00:00, 35.21it/s]


"\nprint(f'名詞\u3000: {noun}')\nprint(f'動詞\u3000: {verb}')\nprint(f'形容詞: {adjv}')\n"

In [52]:
noun = OrderedDict(sorted(noun.items(), key=lambda x:x[1], reverse=True))
verb = OrderedDict(sorted(verb.items(), key=lambda x:x[1], reverse=True))
adjv = OrderedDict(sorted(adjv.items(), key=lambda x:x[1], reverse=True))
noun = dict(itertools.islice(noun.items(), 50))
verb = dict(itertools.islice(verb.items(), 50))
adjv = dict(itertools.islice(adjv.items(), 50))

In [53]:
with open('./test.tsv', 'w') as f:
    w = csv.writer(f, delimiter='\t')
    w.writerow(['word', 'count'])
    for key, val in noun.items():
        w.writerow([key, val])

In [11]:
pd.set_option('display.max_rows', None)
noun.sort_values('count', ascending=False).head()

AttributeError: 'collections.defaultdict' object has no attribute 'sort_values'

In [99]:
verb.sort_values('count', ascending=False).head()

,word,count
7746,し,6366
7737,いる,4311
7747,い,4171
7749,ある,2222
7699,する,1958


In [111]:
adjv.sort_values('count', ascending=False).head()

,word,count
2421,ない,2075
2411,そう,998
2420,どう,680
2415,なく,670
2414,もう,551


In [112]:
noun = noun.sort_values('count', ascending=False).reset_index(drop=True)
verb = verb.sort_values('count', ascending=False).reset_index(drop=True)
adjv = adjv.sort_values('count', ascending=False).reset_index(drop=True)

In [25]:
os.makedirs('./morpheme/front-512/42', exist_ok=True)
noun.to_csv('./morpheme/front-512/42/noun.tsv', sep='\t', index=False)
verb.to_csv('./morpheme/front-512/42/verb.tsv', sep='\t', index=False)
adjv.to_csv('./morpheme/front-512/42/adjv.tsv', sep='\t', index=False)

AttributeError: 'dict' object has no attribute 'to_csv'